Continuamos con las prácticas de Pyspark.
A continuación, crearemos un pequeño dataframe para limpiarlo y hacerle tratamiento de datos.

In [1]:
import findspark

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Spark Profesores").getOrCreate()
spark

In [3]:
from datetime import  datetime, date
import pandas as pd
from pyspark.sql import Row

In [5]:
profesores=[('Ana López', 'Historia', 40), \
    ('Jorge Gómez', 'Matemáticas', 85), \
    ('Jorge Gómez', 'Biología', 35), \
    ('Ana López', 'Literatura', 45), \
    ('Luis García', 'Historia', 30), \
    ('Ana López', 'Historia', 40) \
]
columnas=["Nombre", "Asignatura","Horas"]
df_profesores=spark.createDataFrame(data=profesores, schema=columnas)
df_profesores.show()

+-----------+-----------+-----+
|     Nombre| Asignatura|Horas|
+-----------+-----------+-----+
|  Ana López|   Historia|   40|
|Jorge Gómez|Matemáticas|   85|
|Jorge Gómez|   Biología|   35|
|  Ana López| Literatura|   45|
|Luis García|   Historia|   30|
|  Ana López|   Historia|   40|
+-----------+-----------+-----+



In [6]:
df_profesores.printSchema()

root
 |-- Nombre: string (nullable = true)
 |-- Asignatura: string (nullable = true)
 |-- Horas: long (nullable = true)



In [7]:
#Eliminamos duplicados
df_profesores_distin=df_profesores.distinct()
df_profesores_distin.show()
#Podemos usar .dropDupluicates()

+-----------+-----------+-----+
|     Nombre| Asignatura|Horas|
+-----------+-----------+-----+
|  Ana López|   Historia|   40|
|Jorge Gómez|Matemáticas|   85|
|Jorge Gómez|   Biología|   35|
|  Ana López| Literatura|   45|
|Luis García|   Historia|   30|
+-----------+-----------+-----+



In [8]:
#Eliminamos duplicados en "Asignatura"
dropDisDf=df_profesores.dropDuplicates(["Asignatura"])


In [10]:
dropDisDf.show()


+-----------+-----------+-----+
|     Nombre| Asignatura|Horas|
+-----------+-----------+-----+
|Jorge Gómez|   Biología|   35|
|  Ana López|   Historia|   40|
|  Ana López| Literatura|   45|
|Jorge Gómez|Matemáticas|   85|
+-----------+-----------+-----+



Parte 2

Creamos dos dataframes con PK y FK

In [12]:
profesores2=[(1, 'Ana López', 'Física', 30, 3),(2, 'Marta Rodríguez', 'Química', 35,4),(3, 'Marta Rodríguez', 'Biología', 65,1), (4, 'Marta Rodríguez', 'Matemáticas', 45,3),(5, 'Ana López', 'Literatura', 50,2),(6, 'Isabel Sánchez', 'Física', 40,6)]
columnas2=["Id","Nombre", "Asignatura","Horas", "Id_cole"]

colegio=[(1, 'Escuela La Esperanza', 'Sevilla'), \
    (2, 'Colegio El Prado', 'Sevilla'), \
    (3, 'Colegio San José', 'Bilbao'), \
    (4, 'Colegio San José', 'Sevilla'), \
    (5, 'Colegio San José', 'Barcelona'), \
    (6, 'Instituto Nacional', 'Barcelona') \
    ]
columnas3=["Id_cole", "Colegio", "Ciudad"]
df_profes=spark.createDataFrame(data=profesores2, schema=columnas2)
df_coles=spark.createDataFrame(data=colegio,schema=columnas3)

In [14]:
df_profes.show()

+---+---------------+-----------+-----+-------+
| Id|         Nombre| Asignatura|Horas|Id_cole|
+---+---------------+-----------+-----+-------+
|  1|      Ana López|     Física|   30|      3|
|  2|Marta Rodríguez|    Química|   35|      4|
|  3|Marta Rodríguez|   Biología|   65|      1|
|  4|Marta Rodríguez|Matemáticas|   45|      3|
|  5|      Ana López| Literatura|   50|      2|
|  6| Isabel Sánchez|     Física|   40|      6|
+---+---------------+-----------+-----+-------+



In [15]:
df_coles.show()

+-------+--------------------+---------+
|Id_cole|             Colegio|   Ciudad|
+-------+--------------------+---------+
|      1|Escuela La Esperanza|  Sevilla|
|      2|    Colegio El Prado|  Sevilla|
|      3|    Colegio San José|   Bilbao|
|      4|    Colegio San José|  Sevilla|
|      5|    Colegio San José|Barcelona|
|      6|  Instituto Nacional|Barcelona|
+-------+--------------------+---------+



In [16]:
#RealizAMOS UN INNERJOIN
df_profes.join(df_coles, df_profes.Id_cole == df_coles.Id_cole,"inner").show()

+---+---------------+-----------+-----+-------+-------+--------------------+---------+
| Id|         Nombre| Asignatura|Horas|Id_cole|Id_cole|             Colegio|   Ciudad|
+---+---------------+-----------+-----+-------+-------+--------------------+---------+
|  3|Marta Rodríguez|   Biología|   65|      1|      1|Escuela La Esperanza|  Sevilla|
|  5|      Ana López| Literatura|   50|      2|      2|    Colegio El Prado|  Sevilla|
|  1|      Ana López|     Física|   30|      3|      3|    Colegio San José|   Bilbao|
|  4|Marta Rodríguez|Matemáticas|   45|      3|      3|    Colegio San José|   Bilbao|
|  2|Marta Rodríguez|    Química|   35|      4|      4|    Colegio San José|  Sevilla|
|  6| Isabel Sánchez|     Física|   40|      6|      6|  Instituto Nacional|Barcelona|
+---+---------------+-----------+-----+-------+-------+--------------------+---------+



In [17]:
df_profes.join(df_coles, df_profes.Id_cole == df_coles.Id_cole,"outer").show()

+----+---------------+-----------+-----+-------+-------+--------------------+---------+
|  Id|         Nombre| Asignatura|Horas|Id_cole|Id_cole|             Colegio|   Ciudad|
+----+---------------+-----------+-----+-------+-------+--------------------+---------+
|   3|Marta Rodríguez|   Biología|   65|      1|      1|Escuela La Esperanza|  Sevilla|
|   5|      Ana López| Literatura|   50|      2|      2|    Colegio El Prado|  Sevilla|
|   1|      Ana López|     Física|   30|      3|      3|    Colegio San José|   Bilbao|
|   4|Marta Rodríguez|Matemáticas|   45|      3|      3|    Colegio San José|   Bilbao|
|   2|Marta Rodríguez|    Química|   35|      4|      4|    Colegio San José|  Sevilla|
|NULL|           NULL|       NULL| NULL|   NULL|      5|    Colegio San José|Barcelona|
|   6| Isabel Sánchez|     Física|   40|      6|      6|  Instituto Nacional|Barcelona|
+----+---------------+-----------+-----+-------+-------+--------------------+---------+



In [18]:
df_profes.join(df_coles, df_profes.Id_cole == df_coles.Id_cole,"fullouter").show()

+----+---------------+-----------+-----+-------+-------+--------------------+---------+
|  Id|         Nombre| Asignatura|Horas|Id_cole|Id_cole|             Colegio|   Ciudad|
+----+---------------+-----------+-----+-------+-------+--------------------+---------+
|   3|Marta Rodríguez|   Biología|   65|      1|      1|Escuela La Esperanza|  Sevilla|
|   5|      Ana López| Literatura|   50|      2|      2|    Colegio El Prado|  Sevilla|
|   1|      Ana López|     Física|   30|      3|      3|    Colegio San José|   Bilbao|
|   4|Marta Rodríguez|Matemáticas|   45|      3|      3|    Colegio San José|   Bilbao|
|   2|Marta Rodríguez|    Química|   35|      4|      4|    Colegio San José|  Sevilla|
|NULL|           NULL|       NULL| NULL|   NULL|      5|    Colegio San José|Barcelona|
|   6| Isabel Sánchez|     Física|   40|      6|      6|  Instituto Nacional|Barcelona|
+----+---------------+-----------+-----+-------+-------+--------------------+---------+



In [20]:
df_profes.join(df_coles, df_profes.Id_cole == df_coles.Id_cole,"left").show()

+---+---------------+-----------+-----+-------+-------+--------------------+---------+
| Id|         Nombre| Asignatura|Horas|Id_cole|Id_cole|             Colegio|   Ciudad|
+---+---------------+-----------+-----+-------+-------+--------------------+---------+
|  1|      Ana López|     Física|   30|      3|      3|    Colegio San José|   Bilbao|
|  2|Marta Rodríguez|    Química|   35|      4|      4|    Colegio San José|  Sevilla|
|  3|Marta Rodríguez|   Biología|   65|      1|      1|Escuela La Esperanza|  Sevilla|
|  4|Marta Rodríguez|Matemáticas|   45|      3|      3|    Colegio San José|   Bilbao|
|  5|      Ana López| Literatura|   50|      2|      2|    Colegio El Prado|  Sevilla|
|  6| Isabel Sánchez|     Física|   40|      6|      6|  Instituto Nacional|Barcelona|
+---+---------------+-----------+-----+-------+-------+--------------------+---------+



In [24]:
#funciones definidas por el usuario:
from pyspark.sql.functions import udf
from pyspark.sql.functions import col
def upperCase(str):
    """ PAsamos una string a mayúsculas"""
    return str.upper()

In [22]:
upperCaseUDF=udf(lambda x: upperCase(x))

In [26]:
df_profes.select(col("Nombre"), upperCaseUDF(col("Nombre"))).show()

+---------------+----------------+
|         Nombre|<lambda>(Nombre)|
+---------------+----------------+
|      Ana López|       ANA LÓPEZ|
|Marta Rodríguez| MARTA RODRÍGUEZ|
|Marta Rodríguez| MARTA RODRÍGUEZ|
|Marta Rodríguez| MARTA RODRÍGUEZ|
|      Ana López|       ANA LÓPEZ|
| Isabel Sánchez|  ISABEL SÁNCHEZ|
+---------------+----------------+



In [29]:
df_profes.withColumn("NOMBRE MAYUSCULA", upperCaseUDF(col("Nombre"))).show()

+---+---------------+-----------+-----+-------+----------------+
| Id|         Nombre| Asignatura|Horas|Id_cole|NOMBRE MAYUSCULA|
+---+---------------+-----------+-----+-------+----------------+
|  1|      Ana López|     Física|   30|      3|       ANA LÓPEZ|
|  2|Marta Rodríguez|    Química|   35|      4| MARTA RODRÍGUEZ|
|  3|Marta Rodríguez|   Biología|   65|      1| MARTA RODRÍGUEZ|
|  4|Marta Rodríguez|Matemáticas|   45|      3| MARTA RODRÍGUEZ|
|  5|      Ana López| Literatura|   50|      2|       ANA LÓPEZ|
|  6| Isabel Sánchez|     Física|   40|      6|  ISABEL SÁNCHEZ|
+---+---------------+-----------+-----+-------+----------------+



Parte 3

Tratamos los nulos

In [30]:
profesores4=[(1, 'Ana López', 'Física', None, 3),(2, 'Marta Rodríguez', 'Química', 35,4),(3, 'Marta Rodríguez', None, 65,1), (4, 'Marta Rodríguez', 'Matemáticas', 45,3),(5, 'Ana López', 'Literatura', 50,2),(6, 'Isabel Sánchez', 'Física', 40,6)]
columnas4=["Id","Nombre", "Asignatura","Horas", "Id_cole"]
df_profes_null=spark.createDataFrame(data=profesores4, schema=columnas4)


In [31]:
df_profes_null.show()

+---+---------------+-----------+-----+-------+
| Id|         Nombre| Asignatura|Horas|Id_cole|
+---+---------------+-----------+-----+-------+
|  1|      Ana López|     Física| NULL|      3|
|  2|Marta Rodríguez|    Química|   35|      4|
|  3|Marta Rodríguez|       NULL|   65|      1|
|  4|Marta Rodríguez|Matemáticas|   45|      3|
|  5|      Ana López| Literatura|   50|      2|
|  6| Isabel Sánchez|     Física|   40|      6|
+---+---------------+-----------+-----+-------+



In [32]:
#Usaremos .fill  para rellenar los valores nulos por un "999"
df_profes_null.na.fill(value= 999).show()

+---+---------------+-----------+-----+-------+
| Id|         Nombre| Asignatura|Horas|Id_cole|
+---+---------------+-----------+-----+-------+
|  1|      Ana López|     Física|  999|      3|
|  2|Marta Rodríguez|    Química|   35|      4|
|  3|Marta Rodríguez|       NULL|   65|      1|
|  4|Marta Rodríguez|Matemáticas|   45|      3|
|  5|      Ana López| Literatura|   50|      2|
|  6| Isabel Sánchez|     Física|   40|      6|
+---+---------------+-----------+-----+-------+



In [33]:
df_profes_null.na.fill(value= 999, subset=["Horas"]).show()

+---+---------------+-----------+-----+-------+
| Id|         Nombre| Asignatura|Horas|Id_cole|
+---+---------------+-----------+-----+-------+
|  1|      Ana López|     Física|  999|      3|
|  2|Marta Rodríguez|    Química|   35|      4|
|  3|Marta Rodríguez|       NULL|   65|      1|
|  4|Marta Rodríguez|Matemáticas|   45|      3|
|  5|      Ana López| Literatura|   50|      2|
|  6| Isabel Sánchez|     Física|   40|      6|
+---+---------------+-----------+-----+-------+



In [37]:
df_profes_null.na.fill("Desconocido", ["Asignatura"]).na.fill(0, ["Horas"]).show()
#Tener en cuenta el tipo de valor que se incerta. Debe coincidir con el tipo de valores de la tabla.

+---+---------------+-----------+-----+-------+
| Id|         Nombre| Asignatura|Horas|Id_cole|
+---+---------------+-----------+-----+-------+
|  1|      Ana López|     Física|    0|      3|
|  2|Marta Rodríguez|    Química|   35|      4|
|  3|Marta Rodríguez|Desconocido|   65|      1|
|  4|Marta Rodríguez|Matemáticas|   45|      3|
|  5|      Ana López| Literatura|   50|      2|
|  6| Isabel Sánchez|     Física|   40|      6|
+---+---------------+-----------+-----+-------+



In [38]:
spark.stop()